# Gerador de Dados PIX - Notebook Didático Completo

**Objetivo:** Este notebook apresenta uma versão educativa e detalhada do script de geração de dados sintéticos PIX. O processo é executado passo a passo para um município específico (**São Paulo**) e inclui:

- Explicações claras de cada etapa.
- Fórmulas matemáticas que governam a lógica.
- Visualizações gráficas para analisar os dados.
- Trechos de código que implementam a teoria.

O objetivo final é transformar o processo de geração de dados de uma "caixa-preta" para uma "caixa de vidro", permitindo total compreensão e confiança nos resultados.

In [0]:
%pip install faker

In [0]:
# ==============================================================================
# PASSO 1: IMPORTAÇÕES E CONFIGURAÇÃO INICIAL
# ==============================================================================

# Bibliotecas padrão
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
import pandas as pd
import numpy as np
from faker import Faker
from datetime import datetime, timedelta
import random
from scipy.stats import lognorm
import uuid
import gc

# Bibliotecas para visualização
import matplotlib.pyplot as plt
import seaborn as sns

# Configuração para os gráficos ficarem mais bonitos
sns.set_theme(style="whitegrid", palette="viridis")
print("Bibliotecas importadas com sucesso, incluindo Matplotlib e Seaborn para gráficos.")

# Inicialização do Spark e Faker
spark = SparkSession.builder.appName("PixGenerator_AnaliseDidatica").getOrCreate()
fake = Faker('pt_BR')

# --- PARÂMETROS FIXOS PARA ANÁLISE ---
TARGET_ANO = 2024
TARGET_ESTADO_IBGE = 35
TARGET_MUNICIPIO_IBGE = 3550308
TARGET_REGIAO_ID = 3 # Região Sudeste

POPULATION_SCALING_FACTOR = 1000
DATA_SAMPLE_PERCENTAGE = 0.1 # Usamos 10% dos dados para a análise ser mais rápida
np.random.seed(42)
random.seed(42)
print(f"Análise configurada para o município {TARGET_MUNICIPIO_IBGE} (São Paulo) com {DATA_SAMPLE_PERCENTAGE*100}% da amostra de dados.")

## Passo 2: Carregando e Analisando as Estatísticas de Entrada

Nesta etapa, carregamos os dados estatísticos reais que servem de base para a nossa geração.  
Vamos realizar uma análise exploratória (EDA) em cada tabela para entender a matéria-prima do nosso processo.

In [0]:
try:
    print("Carregando dados do município (relacao_pagadores_recebedores)...")
    relacao_pag_rec_pd = spark.sql(f"""
        SELECT Mes, total_tx_pagador, total_vl_pagador, peso_pagador_nacional, total_tx_pagador_pf
        FROM transacoes_db.pix_baseline_metricas.relacao_pagadores_recebedores
        WHERE ano = {TARGET_ANO} AND cod_ibge_municipio = {TARGET_MUNICIPIO_IBGE}
    """).toPandas()

    print("Carregando perfis de comportamento da região (perfil_de_usuarios)...")
    perfil_de_usuario_pd = spark.sql(f"""
        SELECT id_finalidade, id_formainiciacao, pag_idade, rec_idade, pag_pfpj_id, total_quantidade, ticket_mediana, ticket_p25, ticket_p75, ticket_minimo, ticket_maximo
        FROM transacoes_db.pix_baseline_metricas.perfil_de_usuarios
        WHERE ano = {TARGET_ANO} AND pag_id_regiao = {TARGET_REGIAO_ID} AND rec_id_regiao = {TARGET_REGIAO_ID}
    """).toPandas()
    
    # Carregando tabelas de dimensão para enriquecer os gráficos
    finalidades_pd = spark.table("transacoes_db.copper.finalidade_pix").toPandas().set_index('id')
    print("Dados carregados com sucesso!")
except Exception as e:
    print(f"ERRO ao carregar dados: {e}")
    raise e

2.1 Análise do Volume do Município (relacao_pag_rec_pd)
Inspecionamos a tabela que nos diz o volume total de transações para São Paulo. O gráfico mostra a magnitude e a variação mensal.

In [0]:
print("[ANÁLISE] Tabela 'relacao_pagadores_recebedores' para São Paulo:")
print(relacao_pag_rec_pd)

# Gráfico 1: Volume de Transações Mensais em SP
plt.figure(figsize=(12, 6))
sns.barplot(data=relacao_pag_rec_pd, x='Mes', y='total_tx_pagador', color='skyblue')
plt.title('Volume Total de Transações de Pagadores por Mês em São Paulo (2024)', fontsize=16)
plt.xlabel('Mês', fontsize=12)
plt.ylabel('Total de Transações (em centenas de milhões)', fontsize=12)
plt.show()

## 2.2 Análise dos Perfis de Comportamento (`perfil_de_usuario_pd`)

Agora, investigamos as "receitas" das transações.  
Esta tabela, a nível regional, nos diz como as transações se distribuem por diferentes tipos de comportamento.

In [0]:
print(f"[ANÁLISE] Tabela 'perfil_de_usuarios' para a Região Sudeste (contém {len(perfil_de_usuario_pd)} perfis):")
print(perfil_de_usuario_pd.head())

# Gráfico 2: Distribuição de Perfis por Finalidade
perfil_de_usuario_pd['finalidade_nome'] = perfil_de_usuario_pd['id_finalidade'].map(finalidades_pd['nome'])
plt.figure(figsize=(12, 6))
sns.countplot(data=perfil_de_usuario_pd, y='finalidade_nome', order=perfil_de_usuario_pd['finalidade_nome'].value_counts().index, palette='crest')
plt.title('Distribuição de Perfis por Finalidade da Transação (Região Sudeste)', fontsize=16)
plt.xlabel('Contagem de Perfis', fontsize=12)
plt.ylabel('Finalidade', fontsize=12)
plt.show()

# Gráfico 3: Distribuição do Valor Mediano das Transações entre os Perfis
plt.figure(figsize=(12, 6))
sns.histplot(perfil_de_usuario_pd['ticket_mediana'], bins=50, kde=True, log_scale=True, color='purple')
plt.title('Distribuição do Valor Mediano (Ticket) entre os Perfis (Escala Log)', fontsize=16)
plt.xlabel('Valor Mediano da Transação (R$) - Escala Log', fontsize=12)
plt.ylabel('Contagem de Perfis', fontsize=12)
plt.show()

## Passo 3: Cálculo do Volume de Transações a Gerar

Antes de gerar os dados, precisamos definir quantas transações o script deve criar para o município ao longo de um ano.  
Usamos uma fórmula baseada nos dados estatísticos mensais:

### Fórmula Matemática

<div align="center">

$$
V_{\text{ano}} = (\overline{T}_{\text{mensal}} \times 12) \times P_{\text{amostra}}
$$

</div>

---

### Sumário dos Termos Matemáticos

 Termo                | Explicação para Leigos                                                                                   |
----------------------|---------------------------------------------------------------------------------------------------------|
 **V<sub>ano</sub>**            | Volume anual total de transações a ser gerado para o município.                                         |
 **T̅<sub>mensal</sub>**   | Média do número de transações de pagadores por mês. O traço (T̅) indica "média" dos valores.     |
 **P<sub>amostra</sub>**        | Percentual da amostra (DATA_SAMPLE_PERCENTAGE), ou seja, a fração dos dados reais que queremos simular. |

**Como funciona:**  
Multiplicamos a média mensal (**T̅<sub>mensal</sub>**) por 12 (meses do ano) para obter o total anual.  
Depois, aplicamos o percentual da amostra (**P<sub>amostra</sub>**) para ajustar o volume ao tamanho desejado.

---

A célula de código abaixo implementa esta fórmula.

In [0]:
# T_barra_mensal
volume_medio_mensal_municipio = relacao_pag_rec_pd['total_tx_pagador'].mean()

# V_ano
volume_total_municipio_ano = (volume_medio_mensal_municipio * 12) * DATA_SAMPLE_PERCENTAGE

# Este é o volume de referência da região, usado para calcular as proporções
volume_total_regional_dos_perfis = perfil_de_usuario_pd['total_quantidade'].sum()

print(f"[CÁLCULO] Média de transações mensais em SP (T_mensal_media): {int(volume_medio_mensal_municipio):,}")
print(f"[CÁLCULO] Volume total a ser gerado para SP (V_ano): {int(volume_total_municipio_ano):,}")
print(f"[CÁLCULO] Volume total de referência de todos os perfis da Região Sudeste: {int(volume_total_regional_dos_perfis):,}")

## Passo 4: Geração da População Sintética

Com base no volume de transações, estimamos um número proporcional de clientes sintéticos que realizarão essas transações.

---

### Fórmulas Matemáticas

<div align="center">

$$
N_{\text{clientes}} = \left\lfloor \frac{\overline{T}_{\text{mensal}}}{F_{\text{escala}}} \times P_{\text{amostra}} \right\rfloor \\
N_{\text{PF}} = \lfloor N_{\text{clientes}} \times \%_{\text{PF}} \rfloor \\
N_{\text{PJ}} = N_{\text{clientes}} - N_{\text{PF}}
$$

</div>

---

### Sumário dos Termos Matemáticos

 Termo                | Explicação para Leigos                                                                                   |
----------------------|---------------------------------------------------------------------------------------------------------|
 **N<sub>clientes</sub>**        | Número total de clientes a gerar.                                                              |
 **F<sub>escala</sub>**          | POPULATION_SCALING_FACTOR, fator de escala para estimar clientes a partir do volume de transações. |
 **P<sub>amostra</sub>**         | Percentual da amostra (DATA_SAMPLE_PERCENTAGE), fração dos dados reais que queremos simular.   |
 **%<sub>PF</sub>**              | Proporção de transações de Pessoa Física (PF) no município.                                   |
 **N<sub>PF</sub>**              | Número de clientes Pessoa Física a gerar.                                                     |
 **N<sub>PJ</sub>**              | Número de clientes Pessoa Jurídica a gerar.                                                   |

**Como funciona:**  
Calculamos o número total de clientes a partir do volume médio mensal, ajustando pelo fator de escala e percentual de amostra.  
Depois, dividimos esse total entre PF e PJ conforme a proporção observada no município.

In [0]:
# T_barra_mensal
total_pagadores_tx_medio = relacao_pag_rec_pd['total_tx_pagador'].mean()
total_pagadores_pf_medio = relacao_pag_rec_pd['total_tx_pagador_pf'].mean()

# N_clientes
num_clientes = int((total_pagadores_tx_medio / POPULATION_SCALING_FACTOR) * DATA_SAMPLE_PERCENTAGE)

# %_pf
perc_pf = total_pagadores_pf_medio / total_pagadores_tx_medio

# N_PF e N_PJ
num_clientes_pf = int(num_clientes * perc_pf)
num_clientes_pj = num_clientes - num_clientes_pf

print(f"Com base na média de {int(total_pagadores_tx_medio):,} transações/mês, vamos gerar uma população de:")
print(f" - Clientes Pessoa Física (PF): {num_clientes_pf:,}")
print(f" - Clientes Pessoa Jurídica (PJ): {num_clientes_pj:,}")

# Aqui, o script principal chamaria a função `generate_base_population`.


## Passo 5: A Lógica de Geração das Transações (Explicação Detalhada)

Esta é a etapa mais crucial do processo. Após calcular o volume total de transações a serem geradas para o município (por exemplo, 10 milhões de transações para São Paulo em nossa amostra), precisamos decidir: **como serão essas transações?** Elas devem refletir a variedade e as características do mundo real.

Para isso, usamos a tabela de perfis da região como um "guia de estilos" ou uma "receita de bolo".  
A lógica se divide em duas partes fundamentais:

1. **Quanto de cada ingrediente?** (Distribuição Proporcional)
2. **Como preparar cada ingrediente?** (Geração dos Valores)

---

### 5.1 - Montando a "Receita": Distribuição Proporcional

A ideia principal é fazer com que a "mistura" de tipos de transação no município sintético seja igual à "mistura" observada na região.

A tabela de perfis nos mostra a popularidade de cada tipo de transação. Por exemplo, ela indica que "**PF enviando PIX para PF com chave aleatória**" (**Perfil A**) é muito mais comum do que "**PJ pagando salário para PF**" (**Perfil B**).

Para aplicar isso, calculamos o "**peso**" ou a "**popularidade**" de cada perfil:

<div align="center">

$$
\mathbf{W_i} = \frac{\mathbf{Q_i}}{\sum_{j=1}^{P} \mathbf{Q_j}}
$$

</div>

- **Q<sub>i</sub>**: quantidade de transações do Perfil *i* na região (nosso "ingrediente").
- **∑<sub>j=1</sub><sup>P</sup> Q<sub>j</sub>**: soma das quantidades de todos os perfis.
- **W<sub>i</sub>**: proporção que o Perfil *i* representa do total (ex: 0,15 ou 15%).

Depois, aplicamos essa proporção ao volume total do município:

<div align="center">

$$
\mathbf{V_i} = \lfloor \mathbf{V_{ano}} \times \mathbf{W_i} \rfloor
$$

</div>

- **V<sub>ano</sub>**: volume total de transações a ser gerado (ex: 10 milhões).
- **W<sub>i</sub>**: proporção do perfil.
- **V<sub>i</sub>**: número de transações a serem geradas com as características do Perfil *i* (ex: $10.000.000 \times 15\% = 1.500.000$ transações do tipo A).

---

### 5.2 - Dando "Sabor" a Cada Ingrediente: Geração dos Valores

Agora que sabemos quantas transações de cada tipo serão geradas, precisamos definir o valor monetário de cada uma. Os valores não podem ser todos iguais.

Utilizamos a **Distribuição Log-Normal**, pois ela representa bem o comportamento de dados financeiros: muitas transações pequenas e poucas de valor muito alto.

Para calibrar a distribuição Log-Normal para cada perfil, estimamos seus parâmetros (**μ̂** e **σ̂**) a partir das estatísticas do perfil:

<div align="center">

$$
\mathbf{\hat{\mu}} = \ln(\text{Mediana})
$$

</div>

- **μ̂**: define o "centro" da distribuição.
- A mediana é usada por ser mais robusta a valores extremos do que a média.

<div align="center">

$$
\mathbf{\hat{\sigma}} = \frac{\ln(P_{75}) - \ln(P_{25})}{1.349}
$$

</div>

- **ln(P<sub>75</sub>) - ln(P<sub>25</sub>)**: intervalo interquartil em escala logarítmica, medindo a dispersão dos 50% centrais dos dados.
- Dividimos por **1,349** pois, numa distribuição normal, essa é a distância entre os percentis 25 e 75 em desvios-padrão.

Com **μ̂** e **σ̂** calculados, temos uma "máquina" calibrada para gerar valores realistas para cada perfil de transação.

## Passo 6: Análise dos Dados Sintéticos Gerados (Simulação)

A execução completa da geração pode demorar. Para validar a lógica descrita no Passo 5, vamos simular um DataFrame de saída com **50.000 linhas** e analisar suas características.

Os gráficos abaixo são o que você usaria para verificar a qualidade do seu dataset final.

In [0]:
# ==============================================================================
# PASSO 1: SIMULAÇÃO DE DADOS APRIMORADA
# Geramos um dataset com "assinaturas" mais realistas para cada tipo de fraude.
# ==============================================================================
import matplotlib.pyplot as plt
import seaborn as sns
from faker import Faker
from datetime import datetime
import numpy as np
import pandas as pd
from scipy.stats import lognorm

# Inicialização
fake = Faker('pt_BR')
num_simulado = 200000

# 1. Definir os estados e probabilidades
estados = ['LEGITIMA', 'FRAUDE', 'ANOMALIA_BENIGNA']
probabilidades_estado = [0.95, 0.04, 0.01]
estado_transacao = np.random.choice(estados, size=num_simulado, p=probabilidades_estado)

# 2. Definir os tipos de fraude e suas características
tipos_de_fraude_reais = ['VALOR_ATIPICO', 'TOMADA_DE_CONTA', 'ATAQUE_DE_FREQUENCIA', 'ENGENHARIA_SOCIAL']

# 3. Construir as colunas de forma consistente e com mais detalhes
is_fraud_col, fraud_type_col, valor_col, data_col = [], [], [], []

for estado in estado_transacao:
    data_gerada = fake.date_time_between(start_date=datetime(2024,1,1), end_date=datetime(2024,12,31))
    data_col.append(data_gerada)
    
    if estado == 'LEGITIMA':
        is_fraud_col.append(0)
        fraud_type_col.append('LEGITIMA')
        valor_col.append(np.clip(lognorm.rvs(s=0.8, scale=np.exp(4)), 0.01, 1000))
    elif estado == 'ANOMALIA_BENIGNA':
        is_fraud_col.append(0)
        fraud_type_col.append('ANOMALIA_BENIGNA')
        valor_col.append(np.clip(lognorm.rvs(s=0.5, scale=np.exp(8)), 1000, 20000))
    elif estado == 'FRAUDE':
        is_fraud_col.append(1)
        tipo_fraude_especifico = np.random.choice(tipos_de_fraude_reais)
        fraud_type_col.append(tipo_fraude_especifico)
        
        if tipo_fraude_especifico == 'VALOR_ATIPICO':
            valor_col.append(random.uniform(20000, 80000))
        elif tipo_fraude_especifico == 'TOMADA_DE_CONTA':
            valor_col.append(random.uniform(1000, 5000))
            data_col[-1] = data_gerada.replace(hour=random.randint(1, 4))
        elif tipo_fraude_especifico == 'ATAQUE_DE_FREQUENCIA':
            valor_col.append(random.uniform(10, 100))
        elif tipo_fraude_especifico == 'ENGENHARIA_SOCIAL':
            valor_col.append(random.uniform(500, 3000))

# 4. Criar o DataFrame final e colunas derivadas
transacoes_pd = pd.DataFrame({
    'valor': valor_col,
    'data': data_col,
    'is_fraud': is_fraud_col,
    'fraud_type': fraud_type_col
})
transacoes_pd['hora'] = transacoes_pd['data'].dt.hour
transacoes_pd['mes'] = transacoes_pd['data'].dt.month
transacoes_pd['dia_da_semana'] = transacoes_pd['data'].dt.day_name()

print(f"DataFrame aprimorado gerado com {len(transacoes_pd)} linhas.")
print("Dados de exemplo:")
print(transacoes_pd.head())

In [0]:
# Gráfico 1: Heatmap de Quantidade de Perfis por Finalidade e Tipo de Pagador (PF/PJ)
plt.figure(figsize=(10, 6))
heatmap_data = perfil_de_usuario_pd.pivot_table(
    index='finalidade_nome', 
    columns='pag_pfpj_id', 
    values='total_quantidade', 
    aggfunc='sum', 
    fill_value=0
)
# Rename columns safely
column_map = {1: 'PF', 2: 'PJ'}
heatmap_data = heatmap_data.rename(columns=column_map)
sns.heatmap(heatmap_data, annot=True, fmt='.0f', cmap='crest')
plt.title('Quantidade de Perfis por Finalidade e Tipo de Pagador', fontsize=16)
plt.xlabel('Tipo de Pagador', fontsize=12)
plt.ylabel('Finalidade', fontsize=12)
plt.show()

# Gráfico 2: Boxplot do Ticket Mediano por Finalidade
plt.figure(figsize=(12, 6))
sns.boxplot(data=perfil_de_usuario_pd, x='finalidade_nome', y='ticket_mediana', palette='viridis')
plt.yscale('log')
plt.title('Distribuição do Ticket Mediano por Finalidade (Escala Log)', fontsize=16)
plt.xlabel('Finalidade', fontsize=12)
plt.ylabel('Ticket Mediano (R$) - Log', fontsize=12)
plt.xticks(rotation=30, ha='right')
plt.grid(True, which="both", ls="--")
plt.show()

# Gráfico 3: Distribuição de Idade dos Pagadores por Tipo de Pagador
plt.figure(figsize=(12, 6))
sns.histplot(data=perfil_de_usuario_pd, x='pag_idade', hue='pag_pfpj_id', bins=30, palette='crest', multiple='stack')
plt.title('Distribuição de Idade dos Pagadores por Tipo de Pagador', fontsize=16)
plt.xlabel('Idade do Pagador', fontsize=12)
plt.ylabel('Contagem de Perfis', fontsize=12)
plt.legend(title='Tipo de Pagador', labels=['PF', 'PJ'])
plt.show()


# Análise 1: Distribuição dos Tipos de Fraude (com Nomes Técnicos)

Aqui filtramos apenas as fraudes confirmadas.  
Antes de plotar, mapeamos os nomes internos do script para seus nomes técnicos correspondentes para garantir que o gráfico seja claro e profissional.

In [0]:
# Filtrar APENAS as transações que são fraudes confirmadas
fraudes_confirmadas_df = transacoes_pd[transacoes_pd['is_fraud'] == 1].copy()

# --- NOVO: Mapeamento para nomes técnicos ---
nomes_tecnicos = {
    'VALOR_ATIPICO': 'Valor Atípico (High-Value Anomaly)',
    'TOMADA_DE_CONTA': 'Tomada de Conta (ATO)',
    'ATAQUE_DE_FREQUENCIA': 'Ataque de Frequência (Smurfing)',
    'ENGENHARIA_SOCIAL': 'Eng. Social (Confidence Scam)'
}
fraudes_confirmadas_df['fraud_type'] = fraudes_confirmadas_df['fraud_type'].map(nomes_tecnicos)
# --- FIM DA ALTERAÇÃO ---

fraud_type_counts = fraudes_confirmadas_df['fraud_type'].value_counts()

# Criar o gráfico de rosca
plt.figure(figsize=(10, 8))
colors = sns.color_palette('Reds_r', len(fraud_type_counts))

plt.pie(fraud_type_counts, labels=fraud_type_counts.index, 
        autopct=lambda p: '{:.1f}%\n({:,.0f})'.format(p, p * sum(fraud_type_counts)/100),
        startangle=140, colors=colors,
        wedgeprops={'width': 0.4, 'edgecolor': 'white', 'linewidth': 2},
        textprops={'fontsize': 12})

plt.title('Composição dos Tipos de Fraude Confirmada', fontsize=16, pad=20)
plt.show()

## Análise 2: Assinatura de Valor por Tipo de Fraude

O Box Plot em escala logarítmica é ideal para comparar a distribuição de valores entre os diferentes tipos de fraude, agora identificados com seus nomes técnicos.

In [0]:
plt.figure(figsize=(14, 8))
sns.boxplot(data=fraudes_confirmadas_df, x='fraud_type', y='valor', palette='viridis')
plt.yscale('log')
plt.title('Distribuição de Valor por Tipo de Fraude (Escala Log)', fontsize=16)
plt.xlabel('Tipo de Fraude', fontsize=12)
plt.ylabel('Valor da Transação (R$) - Log', fontsize=12)
# Rotacionar os labels do eixo X para evitar sobreposição, já que os nomes são mais longos
plt.xticks(rotation=15, ha='right')
plt.grid(True, which="both", ls="--")
plt.show()

# Análise 3: O Horário do Crime
 O gráfico de densidade (KDE) nos permite comparar os padrões de atividade ao longo das 24 horas do dia.
A atividade anômala na madrugada, característica de Account Takeover (ATO), deve se destacar claramente.

In [0]:
plt.figure(figsize=(14, 7))
sns.kdeplot(data=transacoes_pd[transacoes_pd['is_fraud'] == 0], x='hora', fill=True, label='Legítima', color='skyblue', common_norm=False, alpha=0.5)
sns.kdeplot(data=transacoes_pd[transacoes_pd['is_fraud'] == 1], x='hora', fill=True, label='Fraude', color='salmon', common_norm=False, alpha=0.6)
plt.title('Distribuição de Transações por Hora do Dia', fontsize=16)
plt.xlabel('Hora do Dia', fontsize=12)
plt.ylabel('Densidade de Ocorrências', fontsize=12)
plt.xticks(range(0, 24))
plt.legend(fontsize=12)
plt.grid(axis='x')
plt.show()

# Análise 4: O Dia Preferido dos Fraudadores
Será que a incidência de fraudes varia significativamente ao longo da semana?

In [0]:
# Ordenar os dias da semana corretamente
dias_ordenados = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
nomes_dias_pt = {'Monday': 'Segunda', 'Tuesday': 'Terça', 'Wednesday': 'Quarta', 'Thursday': 'Quinta', 'Friday': 'Sexta', 'Saturday': 'Sábado', 'Sunday': 'Domingo'}

fraudes_confirmadas_df['dia_da_semana'] = pd.Categorical(fraudes_confirmadas_df['dia_da_semana'], categories=dias_ordenados, ordered=True)
contagem_dias = fraudes_confirmadas_df['dia_da_semana'].value_counts().sort_index()
contagem_dias.index = contagem_dias.index.map(nomes_dias_pt)

plt.figure(figsize=(14, 7))
sns.barplot(x=contagem_dias.index, y=contagem_dias.values, palette='plasma')
plt.title('Contagem de Fraudes por Dia da Semana', fontsize=16)
plt.xlabel('Dia da Semana', fontsize=12)
plt.ylabel('Número de Fraudes', fontsize=12)
plt.show()

# Análise 5: Correlação entre Valor e Hora da Fraude
Finalmente, o gráfico de dispersão nos permite visualizar a relação entre a hora e o valor para cada tipo de fraude, agora com uma legenda clara usando a terminologia técnica.

In [0]:
plt.figure(figsize=(16, 9))
sns.scatterplot(data=fraudes_confirmadas_df, x='hora', y='valor', hue='fraud_type', palette='bright', s=50, alpha=0.7)
plt.yscale('log')
plt.title('Relação entre Hora do Dia e Valor da Fraude', fontsize=16)
plt.xlabel('Hora do Dia', fontsize=12)
plt.ylabel('Valor da Transação (R$) - Log', fontsize=12)
plt.xticks(range(0, 24))
plt.grid(True, which="both", ls="--")
plt.legend(title='Tipo de Fraude', fontsize=11)
plt.show()